<a href="https://colab.research.google.com/github/Elwing-Chou/tibame_0618/blob/main/tibame_0709_ptt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



```
403 Forbidden
1. IP ban: 換ip
2. 模仿的不像: 少了必要的Headers

```



In [ ]:
import urllib.request as req
# 相對路徑
# 1. a.txt: 現在目錄下的a.txt
# 2. ./a.txt: 現在目錄下的a.txt
# 3. ../a.txt: 前一層目錄下的a.txt

def download_imgur_image(url, dirname="."):
    # url = "https://i.imgur.com/OSwXeuz.jpeg"
    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/1")
    resp = req.urlopen(r)
    content = resp.read()
    # 純文字檔案(CSV, txt...): open("a.txt", "w/r", encoding="utf-8")
    # 非純文字檔案(mp3, mp4, pdf, docs....): open("a.jpeg", "wb/rb")
    fp = dirname + "/" + url.split("/")[-1]
    with open(fp, "wb") as f:
        f.write(content)

url = "https://i.imgur.com/OSwXeuz.jpeg"
download_imgur_image(url)

In [39]:
import urllib.request as req
import json
import os
import bs4 as bs


# 抓一篇的時候, 你要先幫我創一個資料夾(M.175....)
# 圖片放在這資料夾
# beauty - M.17 - .json jpeg .png
#        - M.xx
#
# 再試整頁(處理刪除/公告)

def download_page(url, root="."):
    # url = "https://www.ptt.cc/bbs/Beauty/M.1751797464.A.265.html"
    dirname = os.path.join(root, url.split("/")[-1])
    if not os.path.exists(dirname):
        os.makedirs(dirname)

    r = req.Request(url)
    r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/1")
    resp = req.urlopen(r)
    content = resp.read()
    html = bs.BeautifulSoup(content)
    # print(html)

    metas = html.find_all("span", {"class":"article-meta-value"})
    author = metas[0]
    board = metas[1]
    title = metas[2]
    post_time = metas[3]
    push_datas = html.find_all("div", {"class":"push"})


    author_text = author.text.strip()
    board_text = board.text.strip()
    title_text = title.text.strip()
    post_time_text = post_time.text.strip()

    pushes = []
    for p in push_datas:
        push_meta = p.find_all("span")
        # extract
        push_tag = push_meta[0]
        push_uid = push_meta[1]
        push_content = push_meta[2]
        push_ipdatetime = push_meta[3]
        trans = {"推":1, "噓":-1, "→":0}
        # text
        push_data = {
            "push_tag":trans[push_tag.text.strip()],
            "push_uid":push_uid.text.strip(),
            "push_content":push_content.text.strip().replace(": ", ""),
            "push_ipdatetime":push_ipdatetime.text.strip()
        }
        pushes.append(push_data)

    # downlaod image
    allow = ["png", "jpg", "jpeg", "gif", "webp"]
    links = html.find_all("a")
    for link in links:
        href = link["href"]
        sub = href.split(".")[-1]
        if sub.lower() in allow:
            download_imgur_image(href, dirname)

    # 區塊.extract(): 在整份html裡消失
    main_content = html.find("div", {"id":"main-content"})
    for e in main_content.find_all("div", {"class":"article-metaline"}):
        e.extract()
    for e in main_content.find_all("div", {"class":"article-metaline-right"}):
        e.extract()
    for e in main_content.find_all("div", {"class":"push"}):
        e.extract()
    content_text = main_content.text

    row = {
        "author":author_text,
        "board":board_text,
        "title":title_text,
        "post_time":post_time_text,
        "content":content_text,
        "pushes":pushes,
    }
    # print(row)
    fp = os.path.join(dirname, "data.json")
    f = open(fp, "w", encoding="utf-8")
    json.dump(row, f, indent=4, ensure_ascii=False)
    f.close()

# test
url = "https://www.ptt.cc/bbs/Beauty/M.1751797464.A.265.html"
download_page(url, "beauty")

In [ ]:
import os
dirname = "a/b/c/d"
if not os.path.exists(dirname):
    os.makedirs(dirname)

In [43]:
import urllib.request as req
import json
import os
import bs4 as bs

url = "https://www.ptt.cc/bbs/Beauty/index4003.html"
r = req.Request(url)
r.add_header("User-Agent", "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/1")
resp = req.urlopen(r)
content = resp.read()
html = bs.BeautifulSoup(content)
posts = html.find_all("div", {"class":"title"})
for post in posts:
    link = post.find("a")
    if not link == None:
        if not "公告" in link.text:
            post_url = "https://www.ptt.cc" + link["href"]
            print(post_url)
            download_page(post_url, "Beauty")

https://www.ptt.cc/bbs/Beauty/M.1752031632.A.622.html
https://www.ptt.cc/bbs/Beauty/M.1752032195.A.688.html
https://www.ptt.cc/bbs/Beauty/M.1752033614.A.F9B.html
https://www.ptt.cc/bbs/Beauty/M.1752033795.A.7C2.html
https://www.ptt.cc/bbs/Beauty/M.1752036469.A.6E8.html
https://www.ptt.cc/bbs/Beauty/M.1752040197.A.836.html
https://www.ptt.cc/bbs/Beauty/M.1752044396.A.565.html
https://www.ptt.cc/bbs/Beauty/M.1752046109.A.FF7.html


In [46]:
import glob
import pandas as pd
import json

table = []
fp_list = glob.glob("Beauty/*/*.json")
for fp in fp_list:
    with open(fp, "r", encoding="utf-8") as f:
        content = f.read()
        row = json.loads(content)
        table.append(row)
df = pd.DataFrame(table)
df

,author,board,title,post_time,content,pushes
0,Gentlemon (肥宅紳士),Beauty,[正妹] Cosplay 2099 日本 正,Wed Jul 9 12:47:47 2025,\n\n\n\n\n\n1.https://i.imgur.com/hG175nW.jpeg...,"[{'push_tag': 1, 'push_uid': 'bio5chris', 'pus..."
1,Kjartan (Olafsson),Beauty,[正妹] 刺青穿環大奶,Wed Jul 9 11:27:10 2025,\nhttps://images2.imgbox.com/37/2c/ILuQOrW3_o....,"[{'push_tag': 1, 'push_uid': 'penick', 'push_c..."
2,Kjartan (Olafsson),Beauty,[正妹] Vera Branco 葡萄牙女孩,Wed Jul 9 15:28:18 2025,\nVera Branco 葡萄牙女孩\n\nhttps://images2.imgbox...,[]
3,YuiLover (羽冰漾),Beauty,[正妹] yanyan,Wed Jul 9 12:03:13 2025,\nhttps://i.imgur.com/aMBrHex.jpeg\nhttps://i....,"[{'push_tag': 1, 'push_uid': 'sa13961387', 'pu..."
4,yavakaka (爛兵683T),Beauty,[正妹] 語寶,Wed Jul 9 13:49:37 2025,\nhttps://i.postimg.cc/zX7KMC0T/c-ty-R-Downloa...,"[{'push_tag': 1, 'push_uid': 'Krishna', 'push_..."
5,AmedRosario (),Beauty,[正妹] 瑜珈老師,Wed Jul 9 12:00:10 2025,\nhttps://i.meee.com.tw/ThWlQKE.jpg\nhttps://i...,"[{'push_tag': 1, 'push_uid': 'maxxxxxx', 'push..."
6,s7503228 (情報屋),Beauty,[正妹] 可愛的,Wed Jul 9 11:36:32 2025,\nhttps://pbs.twimg.com/media/GrOZimiW8AAQKUx....,[]
7,yavakaka (爛兵683T),Beauty,[正妹] 色氣,Wed Jul 9 14:59:28 2025,\nhttps://i.postimg.cc/yYRc6q1X/S-63553581-0.j...,"[{'push_tag': 0, 'push_uid': 'lovegogi', 'push..."


In [36]:
os.path.join("a/", "b", "c", "d")

'a/b/c/d'

In [ ]:
# os.path json.dump/dumps/requests